In [ ]:
# 1. Export everything
# 2. Make a copy
# 3. Improt it into mongo

# 4. Query with geothings
# Do export maps

In [2]:
# import all 
import os
import requests
import json
from dotenv import load_dotenv
import pandas as pd
import geopandas as gpd
from cartoframes.viz import Map, Layer, popup_element
import json

In [60]:
# connect w API 
where1= 'Singapore, Singapore'
where2= 'Zurich, Switzerland '
where3= 'Atlanta, United States'

# connect w API 
import time
def geo_code (where):
    try:
        url_geocode1 = f"https://geocode.xyz/{where}?json=1"
        res = requests.get(url_geocode1).json()
        return res['latt'], res['longt']
    except:        
        time.sleep(3)
        geo_code()


In [45]:
# Get coordinates for 3 wheres  
def get_coordinates (where):
    url_geocode = f"https://geocode.xyz/{where}?json=1"

    try:
        res = requests.get(url_geocode).json()
        return res["latt"], res["longt"]
    except:
        print(f"Sorry, no matches found for {where}")
    get_coordinates (where)

    

In [46]:
# get Coordinates for each city 
Singapore= get_coordinates ('Singapore, Singapore')
Singapore

('1.33594', '103.83717')

In [47]:
Zurich= get_coordinates ('Zurich, Switzerland')
Zurich

('47.36670', '8.55000')

In [64]:
Atlanta = get_coordinates ('Atlanta, United States of America')
Atlanta

('33.77729', '-84.39015')

In [ ]:
#put coordinates into Type Point
def type_point(c_tuple):
    TP_coords= {'type': 'Point', 'coordinates': c_tuple}
    return TP_coords

In [66]:
Sing= type_point(Singapore)
Zur= type_point(Zurich)
At= type_point(Atlanta)

In [67]:
# Connect w/ Foursquare API 
load_dotenv()
token_fsq = os.getenv("token3")

In [68]:

def get_query(query, coords, limit= 1,radius= 1000):
    """
    Query foursquare around coordinates using outputs from get_coordinates
    """
    lat= coords[0]
    lon= coords[1]
    url = f"https://api.foursquare.com/v3/places/search?query={query}&ll={lat}%2C{lon}&radius={str(radius)}&limit={str(limit)}"
    headers = {
        "accept": "application/json",
        "Authorization": token_fsq}
    response = requests.get(url, headers=headers).json()
    return response


In [69]:
def get_all(output_get_query): 
    """
    Get name, address, and geopoints from each city
    """
    new_list= []
    for i in output_get_query["results"]:
        name = i["name"]
        address =  i["location"]["formatted_address"]
        lat = i["geocodes"]["main"]["latitude"]
        lon = i["geocodes"]["main"]["longitude"]
        category= i['categories'][0]['name']

        type_ = {"typepoint": 
                                {"type": "Point", 
                                "coordinates": [lat, lon]}}
        new_list.append({"name":name, "lat":lat, "lon":lon, "category": category, "type":type_})
    df = pd.DataFrame(new_list)
    return df

 ### Make Map of Qualities in Singapore 


In [70]:
#query API for tech in Singapore
outputsing= get_query('tech',Singapore, limit= 50, radius= 7000)

# Get tech businesses Singapore 
techs= get_all(outputsing)
tech_singapore= techs[techs['category']== 'Technology Business']


In [71]:
# Want to add layer to map of design companies 
desing= get_query('design', Singapore, limit= 50, radius= 7000)
designsing= get_all(desing)


In [106]:
def save_df (df, name):
    df.to_csv(f"{name}.csv")

In [72]:
# Query starbucks in Singapore 
starsing= get_query('Starbucks', Singapore, limit= 50, radius= 7000)
starbucksing= get_all(starsing)


In [108]:
save_df(starbucksing, "starbucksing")

In [73]:
#Query airports Singapore
airsing= get_query('Airport', Singapore, limit= 50, radius= 7000)
airsing= get_all(airsing)


In [109]:
save_df(airsing, "airsing")

In [74]:
# Get all category airports Singapore
AirSingport= airsing[airsing['category']== 'Airport']
AirSingAll= airsing[airsing['category']== 'Airport Service']
AirSing= pd.concat([AirSingport, AirSingAll], ignore_index=True)


In [75]:
#Query Basketball Singapore 
basketsing= get_query('Basketball', Singapore, limit= 50, radius= 7000)

# Drop unwanted Basketball categories 
Basketsing= get_all(basketsing)
BasketSingC= Basketsing[Basketsing['category']!= 'Organization']
BasketSingC.drop(index= 42, axis=1, inplace= True)
BasketSingC.drop(index= 49, axis=1, inplace= True )



/opt/anaconda3/envs/Ironhack/lib/python3.7/site-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [76]:
# get Daycares for Singapore 
caresing= get_query('daycare', Singapore, limit= 50, radius= 7000)


In [77]:
# Drop non-related queries 
caresingg= get_all(caresing)
caresingg.drop(index= 21, axis= 1, inplace= True)
caresingg.drop(index= 15, axis=1, inplace= True)
caresingg.drop(index= 18, axis=1, inplace= True)
caresingg.drop(index= 19, axis=1, inplace= True)
caresingg.drop(index= 22, axis=1, inplace= True)



In [78]:
# Get bars from Singapore 
barsing= get_query('bars', Singapore, limit= 50, radius= 7000)
BarSing= get_all(barsing)



In [79]:
#Get dog groomers from Singapore 
groom= get_query('Pet Service', Singapore, limit= 50, radius= 7000)
groomsing= get_all(groom)

In [80]:
#Drop unrelated categories 
GroomSing= groomsing[groomsing['category']== 'Pet Grooming Service']
GroomSing.reset_index(drop=True, inplace=True)


In [ ]:
# Export your results
# Re-import

# export maps as well and re-import

In [81]:
# Make singapore map with layers 
Sing_geodf = gpd.GeoDataFrame(tech_singapore, geometry=gpd.points_from_xy(tech_singapore["lon"], tech_singapore["lat"]))
Sing_geoStar= gpd.GeoDataFrame(starbucksing, geometry=gpd.points_from_xy(starbucksing["lon"], starbucksing["lat"]))
Sing_geoDes= gpd.GeoDataFrame(designsing, geometry=gpd.points_from_xy(designsing["lon"], designsing["lat"]))
Sing_geoAir= gpd.GeoDataFrame(AirSing, geometry=gpd.points_from_xy(AirSing["lon"], AirSing["lat"]))
Sing_geoBas= gpd.GeoDataFrame(BasketSingC, geometry=gpd.points_from_xy(BasketSingC["lon"], BasketSingC["lat"]))
Sing_geoCare= gpd.GeoDataFrame(caresingg, geometry=gpd.points_from_xy(caresingg["lon"], caresingg["lat"]))
Sing_geoBar= gpd.GeoDataFrame(BarSing, geometry=gpd.points_from_xy(BarSing["lon"], BarSing["lat"]))
Sing_geoGroom= gpd.GeoDataFrame(GroomSing, geometry=gpd.points_from_xy(GroomSing["lon"], GroomSing["lat"]))




In [82]:
#Singapore Map 
Map([
    Layer(Sing_geodf, "color:blue", popup_hover=[popup_element("name")]),
    Layer(Sing_geoStar, 'color:green', popup_hover=[popup_element("name")]),
    Layer(Sing_geoDes, 'color:purple', popup_hover=[popup_element("name")]), 
    Layer(Sing_geoAir, 'color:white', popup_hover=[popup_element("name")]), 
    Layer(Sing_geoBas, 'color:orange', popup_hover=[popup_element("name")]), 
    Layer(Sing_geoCare, 'color:pink', popup_hover=[popup_element("name")]), 
    Layer(Sing_geoBar, 'color:black', popup_hover=[popup_element("name")]), 
    Layer(Sing_geoGroom, 'color:brown', popup_hover=[popup_element("name")])
])


## Map of Atlanta 

In [87]:
#query API for tech in Atlanta 
outputAt=get_query('tech',Atlanta, limit= 50, radius= 5000)

#Get tech businesses Atlanta 
techat= get_all(outputAt)
tech_atlanta= techat[techat['category']== 'Technology Business']


In [88]:
#query API for design in Atlanta 
desat=get_query('design',Atlanta, limit= 50, radius= 5000)

#Get design businesses Atlanta 
desAt= get_all(desat)

#Use Regex to only grab design companies
DesAt= desAt[desAt['category'].str.contains('.*Design.*')==True]


In [89]:
#query API for starbuck in Atlanta 
starat=get_query('starbucks',Atlanta, limit= 50, radius= 5000)

#Get starbucks Atlanta 
StarAt= get_all(starat)


In [90]:
#query API for Basketball in Atlanta 
basat=get_query('Basketball', Atlanta, limit= 30, radius= 5000)

#Get Basketball Atlanta 
basAt= get_all(basat)

#Regex for only relevant output 
BasAt= basAt[basAt['category'].str.contains('.*Basketball.*|.*Stadium|.*Park.*|.*Playground.*')==True]


In [91]:
#query API for Daycares in Atlanta 
atcare=get_query('DayCare', Atlanta, limit= 50, radius= 5000)

#Get Basketball Atlanta 
atCare= get_all(atcare)

#Regex for only relevant output 
AtCare= atCare[atCare['category'].str.contains('.*Community.*|.*Daycare|.*Child Care Service|.*Primary and Secondary School.*')==True]



In [92]:
#query API for Airports in Atlanta 
atair=get_query('Airport', Atlanta, limit= 10, radius= 5000)

#Get Basketball Atlanta 
atAir= get_all(atair)

#Extract for only relevant output 
AtAir= atAir[atAir['category']== 'Airport']


In [93]:
#query API for Bars in Atlanta 
atbar=get_query('bar', Atlanta, limit= 50, radius= 5000)

#Get Basketball Atlanta 
atBar= get_all(atbar)

In [94]:
#query API for dog groomers in Atlanta 
atgroom=get_query('Pet Service', Atlanta, limit= 50, radius= 5000)

#Get Basketball Atlanta 
atGroom= get_all(atgroom)

# isolate Pet Grooming Services 
AtGroom= atGroom[atGroom['category']== 'Pet Grooming Service']
AtGroom.reset_index(inplace= True, drop= True)


In [95]:
# Make geodf for Atlanta map 
At_geotech = gpd.GeoDataFrame(tech_atlanta, geometry=gpd.points_from_xy(tech_atlanta["lon"], tech_atlanta["lat"]))
At_geoDes = gpd.GeoDataFrame(DesAt, geometry=gpd.points_from_xy(DesAt["lon"], DesAt["lat"]))
At_geoStar = gpd.GeoDataFrame(StarAt, geometry=gpd.points_from_xy(StarAt["lon"], StarAt["lat"]))
At_geoBas = gpd.GeoDataFrame(BasAt, geometry=gpd.points_from_xy(BasAt["lon"], BasAt["lat"]))
At_geoCare = gpd.GeoDataFrame(AtCare, geometry=gpd.points_from_xy(AtCare["lon"], AtCare["lat"]))
At_geoAir = gpd.GeoDataFrame(AtAir, geometry=gpd.points_from_xy(AtAir["lon"], AtAir["lat"]))
At_geoBar = gpd.GeoDataFrame(atBar, geometry=gpd.points_from_xy(atBar["lon"], atBar["lat"]))
At_geoGroom = gpd.GeoDataFrame(AtGroom, geometry=gpd.points_from_xy(AtGroom["lon"], AtGroom["lat"]))


In [96]:
# Add layers to map 
Map([
    Layer(At_geotech, "color:blue", popup_hover=[popup_element("name")]),  
    Layer(At_geoDes, "color:purple", popup_hover=[popup_element("name")]),
    Layer(At_geoStar, "color:green", popup_hover=[popup_element("name")]), 
    Layer(At_geoBas, "color:orange", popup_hover=[popup_element("name")]), 
    Layer(At_geoCare, "color:pink", popup_hover=[popup_element("name")]), 
    Layer(At_geoAir, "color:white", popup_hover=[popup_element("name")]), 
    Layer(At_geoBar, "color:black", popup_hover=[popup_element("name")]), 
    Layer(At_geoGroom, "color:brown", popup_hover=[popup_element("name")])
])

### Visualize Zurich

In [114]:
# Query API for tech in Zurich
outputZur= get_query('tech', Zurich, limit= 50, radius= 2000)

# Get tech businesses Zurich
techzur= get_all(outputZur)
tech_Zurich= techzur[techzur['category']== 'Technology Business']


In [115]:
# Query API for design in Zurich
deszur=get_query('design',Zurich, limit= 50, radius= 2000)

# Get design in Zurich
desZur= get_all(deszur)

# Regex Design companies 
DesZur= desZur[desZur['category'].str.contains('.*Design.*')==True]


In [116]:
# Query API for starbucks in Zurich
starZur=get_query('starbucks',Zurich, limit= 50, radius= 2000)

# Get starbucks Zurich
StarZur= get_all(starZur)


In [117]:
# Query API for Basketball in Zurich
zurbas=get_query('basketball',Zurich, limit= 50, radius= 2000)

# Get courts in Zurich
Zurbas= get_all(zurbas)

# Regex Design companies 
ZurBas= Zurbas[Zurbas['category'].str.contains('.*Basketball.*')==True]


In [118]:
# Query API for Daycares in Zurich
zurcare=get_query('daycare',Zurich, limit= 50, radius= 2000)

# Get daycare in Zurich
Zurcare= get_all(zurcare)

# Regex daycares 
ZurCare= Zurcare[Zurcare['category'].str.contains('.*Daycare.*|.*Preschool.*|.*Community and Government.*|.*Nursery School.*')==True]



In [119]:
# Query API for Airports in Zurich
zurair=get_query('airport',Zurich, limit= 50, radius= 2000)

# Get Airports in Zurich
Zurair= get_all(zurair)

# Regex Airports 
ZurAir= Zurair[Zurair['category'].str.contains('.*Airport.*|.*Airport Service.*')==True]
ZurAir.reset_index(inplace= True, drop= True)


In [120]:
# Query API for Bars in Zurich
zurbar=get_query('bar',Zurich, limit= 50, radius= 2000)

# Get Airports in Zurich
Zurbar= get_all(zurbar)



In [121]:
# Query API for Groomers in Zurich
zurgroom=get_query('Pet Service',Zurich, limit= 50, radius= 2000)

# Get Groomers in Zurich
Zurgroom= get_all(zurgroom)

# Filter for Pet Grooming Service 
ZurGroom= Zurgroom[Zurgroom['category']== 'Pet Grooming Service']
ZurGroom.reset_index(inplace= True, drop= True)


In [122]:
# Make geodf for each condition in Zurich 
Zur_geotech = gpd.GeoDataFrame(tech_Zurich, geometry=gpd.points_from_xy(tech_Zurich["lon"], tech_Zurich["lat"]))
Zur_geoDes = gpd.GeoDataFrame(DesZur, geometry=gpd.points_from_xy(DesZur["lon"], DesZur["lat"]))
Zur_geoStar = gpd.GeoDataFrame(StarZur, geometry=gpd.points_from_xy(StarZur["lon"], StarZur["lat"]))
Zur_geoBas = gpd.GeoDataFrame(ZurBas, geometry=gpd.points_from_xy(ZurBas["lon"], ZurBas["lat"]))
Zur_geoCare = gpd.GeoDataFrame(ZurCare, geometry=gpd.points_from_xy(ZurCare["lon"], ZurCare["lat"]))
Zur_geoAir = gpd.GeoDataFrame(ZurAir, geometry=gpd.points_from_xy(ZurAir["lon"], ZurAir["lat"]))
Zur_geoBar = gpd.GeoDataFrame(Zurbar, geometry=gpd.points_from_xy(Zurbar["lon"], Zurbar["lat"]))
Zur_geoGroom = gpd.GeoDataFrame(ZurGroom, geometry=gpd.points_from_xy(ZurGroom["lon"], ZurGroom["lat"]))


In [178]:
# Add layers to map 
Map([
    Layer(Zur_geotech, "color:blue", popup_hover=[popup_element("name")]),
    Layer(Zur_geoDes, "color:purple", popup_hover=[popup_element("name")]),
    Layer(Zur_geoStar, "color:green", popup_hover=[popup_element("name")]), 
    #Layer(Zur_geoBas, "color:orange", popup_hover=[popup_element("name")]), 
    Layer(Zur_geoCare, "color:pink", popup_hover=[popup_element("name")]), 
    Layer(Zur_geoAir, "color:white", popup_hover=[popup_element("name")]), 
    Layer(Zur_geoBar, "color:black", popup_hover=[popup_element("name")]),
    Layer(Zur_geoGroom, "color:brown", popup_hover=[popup_element("name")])
])

In [126]:
# Match categories from each query for conversion to json 
DesZur['category'] = DesZur['category'].str.replace('Website Designer','Design Studio')

def fix_star(StarZur):
    """
    Make Starbucks categories equal for later use in Mongo
    """
    StarZur['category'] = StarZur['category'].str.replace('Café','Coffee Shop')
    StarZur['category'] = StarZur['category'].str.replace('Bakery','Coffee Shop')
    StarZur['category'] = StarZur['category'].str.replace('Bistro','Coffee Shop')
    return StarZur
    
ZurCare['category'] = ZurCare['category'].str.replace('Preschool','Daycare')

ZurAir['category'] = ZurAir['category'].str.replace('Airport Service','Airport')


def fix_bars(Zurbar): 
    """
    Make Bar categories equal for later use in Mongo
    """
    Zurbar['category'] = Zurbar['category'].str.replace('Cocktail Bar','Bar')
    Zurbar['category'] = Zurbar['category'].str.replace('Restaurant','Bar')
    Zurbar['category'] = Zurbar['category'].str.replace('Lounge','Bar')
    Zurbar['category'] = Zurbar['category'].str.replace('Music Venue','Bar')
    Zurbar['category'] = Zurbar['category'].str.replace('Cocktail Bar','Bar')
    Zurbar['category'] = Zurbar['category'].str.replace('Concert Hall','Bar')
    Zurbar['category'] = Zurbar['category'].str.replace('Hotel Bar','Bar')
    Zurbar['category'] = Zurbar['category'].str.replace('Pub','Bar')
    Zurbar['category'] = Zurbar['category'].str.replace('Wine Bar','Bar')
    Zurbar['category'] = Zurbar['category'].str.replace('Dining and Drinking','Bar')
    return Zurbar




# Export Zurich df 
tech_Zurich
DesZur
StarZur
ZurCare
ZurAir
Zurbar
ZurGroom

ZurichConditions = pd.concat([tech_Zurich, DesZur, StarZur, ZurBas,ZurCare,ZurAir,Zurbar,ZurGroom])
ZurichConditions.reset_index(inplace= True, drop= True)

/opt/anaconda3/envs/Ironhack/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/anaconda3/envs/Ironhack/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/opt/anaconda3/envs/Ironhack/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See th

In [110]:
ZurichConditions
ZurichConditions.to_csv('ZurichConditions.csv', index = False, encoding='utf-8') # False: not include index
#ZurichConditions.to_json ()

In [127]:
ZurichConditions

,name,lat,lon,category,type,geometry
0,TallyFox Social Technologies AG,47.369714,8.545559,Technology Business,"{'typepoint': {'type': 'Point', 'coordinates':...",POINT (8.54556 47.36971)
1,DCTRL — Studio For Creative Technology,47.380275,8.536201,Technology Business,"{'typepoint': {'type': 'Point', 'coordinates':...",POINT (8.53620 47.38027)
2,Dpi Publishing,47.366376,8.549064,Technology Business,"{'typepoint': {'type': 'Point', 'coordinates':...",POINT (8.54906 47.36638)
3,Quentiq,47.364702,8.549989,Technology Business,"{'typepoint': {'type': 'Point', 'coordinates':...",POINT (8.54999 47.36470)
4,Semacle AG,47.364528,8.549349,Technology Business,"{'typepoint': {'type': 'Point', 'coordinates':...",POINT (8.54935 47.36453)
...,...,...,...,...,...,...
181,Restaurant Nelson,47.375889,8.539936,Lounge,"{'typepoint': {'type': 'Point', 'coordinates':...",POINT (8.53994 47.37589)
182,Razzia,47.360833,8.551481,Restaurant,"{'typepoint': {'type': 'Point', 'coordinates':...",POINT (8.55148 47.36083)
183,Hundesalon Top Dog,47.361670,8.550758,Pet Grooming Service,"{'typepoint': {'type': 'Point', 'coordinates':...",POINT (8.55076 47.36167)
184,Petits Amis Tierbedarfsladen,47.361763,8.559924,Pet Grooming Service,"{'typepoint': {'type': 'Point', 'coordinates':...",POINT (8.55992 47.36176)


In [138]:
ZurichConditions.to_json("ZurichCondistions.json", orient= 'records', default_handler= str)

,name,lat,lon,category,type,geometry
0,TallyFox Social Technologies AG,47.369714,8.545559,Technology Business,"{'typepoint': {'type': 'Point', 'coordinates':...",POINT (8.54556 47.36971)
1,DCTRL — Studio For Creative Technology,47.380275,8.536201,Technology Business,"{'typepoint': {'type': 'Point', 'coordinates':...",POINT (8.53620 47.38027)
2,Dpi Publishing,47.366376,8.549064,Technology Business,"{'typepoint': {'type': 'Point', 'coordinates':...",POINT (8.54906 47.36638)
3,Quentiq,47.364702,8.549989,Technology Business,"{'typepoint': {'type': 'Point', 'coordinates':...",POINT (8.54999 47.36470)
4,Semacle AG,47.364528,8.549349,Technology Business,"{'typepoint': {'type': 'Point', 'coordinates':...",POINT (8.54935 47.36453)
...,...,...,...,...,...,...
181,Restaurant Nelson,47.375889,8.539936,Lounge,"{'typepoint': {'type': 'Point', 'coordinates':...",POINT (8.53994 47.37589)
182,Razzia,47.360833,8.551481,Restaurant,"{'typepoint': {'type': 'Point', 'coordinates':...",POINT (8.55148 47.36083)
183,Hundesalon Top Dog,47.361670,8.550758,Pet Grooming Service,"{'typepoint': {'type': 'Point', 'coordinates':...",POINT (8.55076 47.36167)
184,Petits Amis Tierbedarfsladen,47.361763,8.559924,Pet Grooming Service,"{'typepoint': {'type': 'Point', 'coordinates':...",POINT (8.55992 47.36176)


### GeoQuerries 

In [10]:
import pymongo
import requests
from pymongo import MongoClient
from pymongo import GEOSPHERE
import pandas as pd
import json

In [140]:
# Want to make geoquerries for each condition in Zurich 

client = pymongo.MongoClient("localhost:27017")

db = client.get_database("Ironhack")
zcond= db.get_collection("Zur_Cond")

In [141]:
my_location = [47.364702,8.549989] 

In [142]:
# Put location into TypePoint 
def type_point (list_):
    return {"type": "Point", "coordinates":my_location }
my_location_point= type_point(my_location)

In [143]:
my_location_point

{'type': 'Point', 'coordinates': [47.364702, 8.549989]}

In [144]:
zcond.find_one()

{'_id': ObjectId('635f9b9e32be8feba7cbf095'),
 'name': 'TallyFox Social Technologies AG',
 'lat': 47.369714,
 'lon': 8.545559,
 'category': 'Technology Business',
 'type': {'typepoint': {'type': 'Point',
   'coordinates': [47.369714, 8.545559]}},
 'geometry': 'POINT (8.545559 47.369714)'}

In [149]:
#proy_ = {"name":1, "_id": 0}
zcond.find_one({"type.typepoint":
                    {"$geoIntersects": 
                        {"$geometry": my_location_point}}})['name']

'Quentiq'

In [155]:
def find_neighborhood (list_):
    """
    Find business in area
    """
    type_point_converted = type_point(list_)

    proy_ = {"name":1, "_id": 0}
    return zcond.find_one({"type.typepoint":
                        {"$geoIntersects": 
                            {"$geometry": type_point_converted}}}, proy_)

In [156]:
find_area(['47.364702', '8.549989'])

{'name': 'Quentiq'}

### GeoQuerries for Singapore 

In [ ]:
def Drop_Des_Sing(): 
    """
    Drop unrelated categories from Design Singapore Query 
    """
    designsing.drop(index=7, axis=1, inplace=True) 
    designsing.drop(index=8, axis=1, inplace=True) 
    designsing.drop(index=12, axis=1, inplace=True) 
    designsing.drop(index=41, axis=1, inplace=True) 
    return designsing


In [174]:
def Cats_Des_Sing(designsing):
    """
    Match categories for Design in Singapore 
    """
    designsing['category'] = designsing['category'].str.replace("Children's Clothing Store",'Design Studio')
    designsing['category'] = designsing['category'].str.replace("Women's Store",'Design Studio')
    designsing['category'] = designsing['category'].str.replace("Construction",'Design Studio')
    designsing['category'] = designsing['category'].str.replace("General Contractor",'Design Studio')
    designsing['category'] = designsing['category'].str.replace("Interior Designer",'Design Studio')
    designsing['category'] = designsing['category'].str.replace("Arts and Entertainment",'Design Studio')
    designsing['category'] = designsing['category'].str.replace("Art Gallery",'Design Studio')
    designsing['category'] = designsing['category'].str.replace("Retail",'Design Studio')
    designsing['category'] = designsing['category'].str.replace("Clothing Store",'Design Studio')
    designsing['category'] = designsing['category'].str.replace("Gift Store",'Design Studio')
    designsing['category'] = designsing['category'].str.replace("Photographer",'Design Studio')
    designsing['category'] = designsing['category'].str.replace("Carpenter",'Design Studio')
    designsing['category'] = designsing['category'].str.replace("Business and Professional Services",'Design Studio')
    designsing.reset_index(drop= True, inplace= True)
    return designsing

designsing= Cats_Des_Sing(designsing)

In [158]:
tech_singapore
designsing
starbucksing
AirSing
BasketSingC
caresingg
BarSing
GroomSing


#SingConditions = pd.concat([tech_singapore, designsing, starbucksing, BasketSingC,caresingg,AirSing,BarSing,GroomSing])
#SingConditions.reset_index(inplace= True, drop= True)



,name,lat,lon,category,type,geometry
0,Very Important Pets Pawlour,1.336287,103.848363,Pet Grooming Service,"{'typepoint': {'type': 'Point', 'coordinates':...",POINT (103.84836 1.33629)
1,Mad About Grooming,1.321322,103.852650,Pet Grooming Service,"{'typepoint': {'type': 'Point', 'coordinates':...",POINT (103.85265 1.32132)
2,Fluffy Petcare,1.361468,103.852126,Pet Grooming Service,"{'typepoint': {'type': 'Point', 'coordinates':...",POINT (103.85213 1.36147)
3,Animal Arts Academy at Serangoon Garden,1.364886,103.866020,Pet Grooming Service,"{'typepoint': {'type': 'Point', 'coordinates':...",POINT (103.86602 1.36489)
4,Happy Dog,1.331116,103.796097,Pet Grooming Service,"{'typepoint': {'type': 'Point', 'coordinates':...",POINT (103.79610 1.33112)
5,Surrpaws,1.345625,103.882531,Pet Grooming Service,"{'typepoint': {'type': 'Point', 'coordinates':...",POINT (103.88253 1.34563)
6,A Dog's Life Pet Salon,1.283512,103.834542,Pet Grooming Service,"{'typepoint': {'type': 'Point', 'coordinates':...",POINT (103.83454 1.28351)
7,The Grooming Table,1.287245,103.813379,Pet Grooming Service,"{'typepoint': {'type': 'Point', 'coordinates':...",POINT (103.81338 1.28724)
8,Petopia,1.383385,103.875599,Pet Grooming Service,"{'typepoint': {'type': 'Point', 'coordinates':...",POINT (103.87560 1.38339)
9,Dollhouse Pets,1.277195,103.836846,Pet Grooming Service,"{'typepoint': {'type': 'Point', 'coordinates':...",POINT (103.83685 1.27720)
